In [16]:
import pandas as pd
import os
import pydub
from pydub import AudioSegment
import math

In [13]:
df = pd.read_csv('/om/user/arjunp/goldTranscripts/Lily_040002_first5.csv')
rawaudio = '/om/user/arjunp/rawAudioFiles/Lily_040002_first5.wav'
train_folder = '/om/user/arjunp/fineTuningDataFolders/alexChunkedData/data/train'
test_folder = '/om/user/arjunp/fineTuningDataFolders/alexChunkedData/data/test'
datadir= '/om/user/arjunp/fineTuningDataFolders/alexChunkedData/data'

In [5]:
df

,speaker_code,speaker_role,text,media_start,media_end
0,MOT,Mother,alright we got a big book here huh,0.000,2.555
1,CHI,Target_Child,mhm,2.555,3.306
2,MOT,Mother,okay,3.306,4.695
3,MOT,Mother,great who lives in New_York,4.695,6.903
4,CHI,Target_Child,uh,6.903,7.660
...,...,...,...,...,...
108,MOT,Mother,where's the little black bulldog I wonder,288.413,290.325
109,CHI,Target_Child,it's not in this picture,290.325,294.148
110,MOT,Mother,xxx have you and Daddy already discussed this,NaN,NaN
111,MOT,Mother,xxx,294.148,295.900


In [18]:
length_of_df = len(df)
filepathlist = []
transcriptlist = []
audio = AudioSegment.from_file(rawaudio)

for i in range(0,length_of_df, 4): #every 4 utterances go into one chunk; may be mixed speaker; separating into test and train here
    
    if i+3>=length_of_df:
        break

    #starting_index = i
    

    if math.isnan(df.iloc[i]['media_start']):
        i = i+1

    ending_index = i+3

    if math.isnan(df.iloc[ending_index]['media_end']):
        ending_index +=1

    start_ms = df.iloc[i]['media_start']*1000
    end_ms = df.iloc[ending_index]['media_end']*1000
    chunk = audio[start_ms:end_ms]
    
    if i % 5 == 0:
        export_folder = test_folder
        split = 'test'
    else:
        export_folder = train_folder
        split = 'train'

    chunk.export(export_folder+"/"+f"chunk_{i}.wav", format="wav")

    transcription = ""

    for row in range(i, ending_index+1):
        text = df.iloc[row]['text']
        transcription = transcription + " " + text

    #transcription = df.iloc[i]['text'] + " " + df.iloc[i+1]['text'] + " " + df.iloc[i+2]['text'] + " " + df.iloc[i+3]['text']
    filepathlist.append("data/"+split+"/"+f"chunk_{i}.wav")
    transcriptlist.append(transcription)

transcriptdf = pd.DataFrame(list(zip(filepathlist, transcriptlist)), columns =['file_name', 'transcription'])
transcriptdf.to_csv('/om/user/arjunp/fineTuningDataFolders/alexChunkedData/metadata.csv', index=False)
